# GPT API를 활용한 좋은 리뷰요약

In [50]:
import os
from openai import OpenAI
from dotenv import load_dotenv

# 환경변수 로딩
load_dotenv('.env', override=True)

# 메모리에 로딩된 값을 api_key 변수에 대입
OPENAI_API_KEY = os.environ.get("OPENAI_API_KEY")

client = OpenAI(api_key=OPENAI_API_KEY)

completion = client.chat.completions.create(
    model='gpt-3.5-turbo-0125',
    messages=[{'role': 'user', 'content': 'hi'}],
    temperature=0.0
)

print(completion.choices[0].message.content)

Hello! How can I assist you today?


## 데이터 전처리 함수 작성


리뷰 JSON 파일 로드

In [51]:
import json

with open('./res/1.gangneung_mare.json', 'r') as f:
    review_list = json.load(f)

review_list[:3]

[{'review': '바닷가 바로앞이라 너무좋아요.\n객실도 깨끗하고, 샤워기 수압좋고 따뜻한물 잘 나오는것도 좋았어요~ 친구들이랑 왔다가 다들 만족하고 갑니다ㅎ',
  'stars': 5,
  'date': '2일 전'},
 {'review': '오션뷰라 진짜 좋았어요!! ㅎㅎㅎ\n제 생일이라 남자친구하고 처음 오션뷰 예약해서 온거였는데 진짜 색달랐어요. 옆에 바다보면서 닭강정먹고.. 또 새벽에 바다 구경하고.. 진짜 잘 보내다 가요🥰',
  'stars': 5,
  'date': '4일 전'},
 {'review': '황금연휴에 방문했는데 오션뷰가 정말 예쁘더라고요👍\n저는 2층에 머물렀는데, 창밖으로 보이는 바다가 너무 아름다웠어요\n직원분들도 모두 친절하셔서 기분 좋게 머물다 갑니다👏\n다음에도 꼭 다시 방문하고 싶어요!',
  'stars': 5,
  'date': '2025.05.06'}]

좋은 평점 및 나쁜 평점 정의
- 좋은 평점 = 별 5개 이상
- 나쁜 평점 = 별 4개 이하

In [52]:
good_cnt, bad_cnt = 0, 0
for r in review_list:
    if r['stars'] == 5:
        good_cnt += 1
    else:
        bad_cnt += 1

good_cnt, bad_cnt

(352, 68)

좋은 리뷰 및 나쁜 리뷰로 구분 후 저장

In [53]:
reviews_good, reviews_bad = [], []
for r in review_list:
    if r['stars'] == 5:
        # specital token 추가
        reviews_good.append('[REVIEW_START]' + r['review'] + '[REVIEW_END]')
    else:
        # specital token 추가
        reviews_bad.append('[REVIEW_START]' + r['review'] + '[REVIEW_END]')

reviews_good[:3]

['[REVIEW_START]바닷가 바로앞이라 너무좋아요.\n객실도 깨끗하고, 샤워기 수압좋고 따뜻한물 잘 나오는것도 좋았어요~ 친구들이랑 왔다가 다들 만족하고 갑니다ㅎ[REVIEW_END]',
 '[REVIEW_START]오션뷰라 진짜 좋았어요!! ㅎㅎㅎ\n제 생일이라 남자친구하고 처음 오션뷰 예약해서 온거였는데 진짜 색달랐어요. 옆에 바다보면서 닭강정먹고.. 또 새벽에 바다 구경하고.. 진짜 잘 보내다 가요🥰[REVIEW_END]',
 '[REVIEW_START]황금연휴에 방문했는데 오션뷰가 정말 예쁘더라고요👍\n저는 2층에 머물렀는데, 창밖으로 보이는 바다가 너무 아름다웠어요\n직원분들도 모두 친절하셔서 기분 좋게 머물다 갑니다👏\n다음에도 꼭 다시 방문하고 싶어요![REVIEW_END]']

하나의 문자열로 합치기

In [54]:
# good 문자, 하나의 문자열로 합치기
reviews_good_text = '\n'.join(reviews_good)
reviews_good_text[:100]

'[REVIEW_START]바닷가 바로앞이라 너무좋아요.\n객실도 깨끗하고, 샤워기 수압좋고 따뜻한물 잘 나오는것도 좋았어요~ 친구들이랑 왔다가 다들 만족하고 갑니다ㅎ[REVIEW_E'

In [55]:
# bad 문자, 하나의 문자열로 합치기
reviews_bad_text = '\n'.join(reviews_bad)
reviews_bad_text[:100]

'[REVIEW_START]우선 바람이 불어서 예정보다 30분 숙소 입실을 허락해줌\n그런데 오션뷰 2층 205호는 앞에 가로수들때문에 오션전혀 기대하시면 안됩니다. 그리고 침구는 발'

리뷰데이터 전처리를 위한 함수 작성

In [56]:
import datetime
from dateutil import parser

def preprocess_reviews(path):
    with open(path, 'r', encoding='utf-8') as f:
        # list 형태로 json 파일 일기
        review_list = json.load(f)

    # good, bad 데이터 저장 변수 초기화
    reviews_good, reviews_bad = [], []

    # 현재 이전 6개월 데이터 사용을 위해 날짜계산
    current_date = datetime.datetime.now()
    date_boundary = current_date - datetime.timedelta(days=6*30)
    # print(date_boundary)

    # 리뷰데이터 
    for r in review_list:
        review_date_str = r['date']
        try:
            # 리뷰 날짜 문자열을 datetime 객체로 변환
            review_date = parser.parse(review_date_str)
        except (ValueError, TypeError):
            # 정상 날짜 데이터 아닌경우 현재 날짜 대입
            review_date = current_date

        # 날짜 바운더리 보다 작으면, pass
        if review_date < date_boundary:
            continue

        # 평점 5점 기준
        if r['stars'] == 5:
            reviews_good.append('[REVIEW_START]' + r['review'] + '[REVIEW_END]')
        else:
            reviews_bad.append('[REVIEW_START]' + r['review'] + '[REVIEW_END]')

    # good, bad 하나의 문자열 리스트로 합치기
    reviews_good_text = '\n'.join(reviews_good)
    reviews_bad_text = '\n'.join(reviews_bad)

    # good 리뷰와 bad 리뷰 반환
    return reviews_good_text, reviews_bad_text

# 리뷰 데이터 전처리 함수 호출
data='./res/1.gangneung_mare.json'
good, bad = preprocess_reviews(data)
good[:100]

'[REVIEW_START]바닷가 바로앞이라 너무좋아요.\n객실도 깨끗하고, 샤워기 수압좋고 따뜻한물 잘 나오는것도 좋았어요~ 친구들이랑 왔다가 다들 만족하고 갑니다ㅎ[REVIEW_E'

In [57]:
bad[:100]

'[REVIEW_START]우선 바람이 불어서 예정보다 30분 숙소 입실을 허락해줌\n그런데 오션뷰 2층 205호는 앞에 가로수들때문에 오션전혀 기대하시면 안됩니다. 그리고 침구는 발'

### 평가용 함수 작성

평가 기준 설정
- MT-Bench 논문 기반 Pairwise Comparision (=LLM 기반 평가)
  - 비교하는 방식 vs. 점수 매기는 방식
  - 점수라는게 애매 할 수 있음 (ex. 어느 정도의 요약 품질이 3점인가?)
  - 경험상 점수보다는 비교가 상대적으로 더 정확한 편

평가 스크립트 작성
- MT-Bench 논문 Prompt에서 일부 단어만 수정 (ex. Korean summaries, accommodations)
    - [[A]] – 어시스턴트 A가 더 나을 경우
    - [[B]] – 어시스턴트 B가 더 나을 경우
    - [[C]] – 두 응답이 동등한 품질일 경우

In [58]:
# def pairwise_eval(리뷰원본, AI_a, AI_b):
def pairwise_eval(reviews, answer_a, answer_b):
    # 평가 프롬프트 
    eval_prompt = f"""[System]
Please act as an impartial judge and evaluate the quality of the Korean summaries provided by two
AI assistants to the set of user reviews on accommodations displayed below. You should choose the assistant that
follows the user's instructions and answers the user's question better. Your evaluation
should consider factors such as the helpfulness, relevance, accuracy, depth, creativity,
and level of detail of their responses. Begin your evaluation by comparing the two
responses and provide a short explanation. Avoid any position biases and ensure that the
order in which the responses were presented does not influence your decision. Do not allow
the length of the responses to influence your evaluation. Do not favor certain names of
the assistants. Be as objective as possible. After providing your explanation, output your
final verdict by strictly following this format: "[[A]]" if assistant A is better, "[[B]]"
if assistant B is better, and "[[C]]" for a tie.
[User Reviews]
{reviews}

[The Start of Assistant A's Answer]
{answer_a}
[The End of Assistant A's Answer]

[The Start of Assistant B's Answer]
{answer_b}
[The End of Assistant B's Answer]"""
    
    # OpenAI API를 사용하여 평가 요청
    completion = client.chat.completions.create(
        model='gpt-4o-2024-05-13',
        messages=[{'role': 'user', 'content': eval_prompt}],
        temperature=0.0
    )

    return completion.choices[0].message.content

### Baseline 응답 개발

In [59]:
PROMPT_BASELINE = f"""아래 숙소 리뷰에 대해 5문장 내로 요약해줘:"""

In [60]:
# 높은 평점만 리뷰 데이터만 받음음
reviews, _ = preprocess_reviews(path='./res/1.gangneung_mare.json')

### 리뷰 데이터 요약 함수 정의 및 실행

In [61]:
def summarize(reviews, prompt, temperature=0.0, model='gpt-3.5-turbo-0125'):
    prompt = prompt + '\n\n' + reviews

    completion = client.chat.completions.create(
        model=model,
        messages=[{'role': 'user', 'content': prompt}],
        temperature=temperature
    )

    return completion.choices[0].message.content

print(summarize(reviews, PROMPT_BASELINE))

바닷가 바로앞에 위치한 숙소로, 객실이 깨끗하고 샤워기 수압이 좋아서 만족스러웠다. 오션뷰가 좋아서 친구들과 함께 와서 만족했고, 직원들도 친절했다. 위치가 좋고 주차도 편리하며, 숙소 내부가 깔끔하고 아늑했다. 또한, 오션뷰가 아름다워서 편안한 휴식을 취할 수 있었다.


## 응답결과 평가하기

대조군 문장 정의하기

In [62]:
# 대조군 문장 가져오기(야놀자 AI good 요약 문장)
# summary_real_20250530 = '숙소는 강릉 경포해변 바로 앞에 위치하며, 오션뷰와 경포호수 전망을 제공합니다. 객실과 화장실이 청결하게 관리되고 있으며, 편안한 침대와 고급 침구로 쾌적한 숙박 환경을 제공합니다. 주변에 식당과 편의시설이 가까워 접근성이 좋고, 주차 공간도 넓습니다. 직원들의 친절한 응대와 서비스도 긍정적인 평가를 받고 있으며, 대체로 소음 차단이 잘 이루어져 조용한 환경에서 휴식을 취할 수 있습니다. 전반적으로 방문자들은 숙소의 청결도와 편안함, 아늑한 분위기에 매우 만족하는 후기를 남기고 있습니다.'

summary_4o_20250530 = summarize(reviews, PROMPT_BASELINE, temperature=0.0, model='gpt-4o-2024-05-13')
summary_4o_20250530

'이 숙소는 바다와 가까운 위치와 아름다운 오션뷰로 많은 이용객들에게 호평을 받고 있습니다. 객실은 매우 청결하고, 침구와 화장실 상태도 좋다는 평가가 많습니다. 직원들의 친절한 서비스와 편리한 주차 시설도 긍정적인 리뷰를 받았습니다. 다만, 일부 이용객들은 밤에 소음이 있거나 난방이 부족하다는 점을 지적했습니다. 전반적으로 재방문 의사가 높고, 가성비 좋은 숙소로 추천받고 있습니다.'

In [63]:
# 비교하기
# pairwise_eval(리뷰 데이터, AI 요약한 데이터_A, 대조군 AI 문장_B))
print(pairwise_eval(reviews, summarize(reviews, PROMPT_BASELINE), summary_4o_20250530))

Both Assistant A and Assistant B provided summaries that capture the essence of the user reviews, highlighting the positive aspects of the accommodations such as the proximity to the beach, cleanliness, friendly staff, and good views. However, there are some differences in their responses:

1. **Helpfulness and Relevance**:
   - **Assistant A**: Focuses on the positive aspects and mentions specific details like the shower pressure and the popularity among different types of guests (e.g., friends, couples).
   - **Assistant B**: Provides a more balanced view by also mentioning some negative aspects like noise and heating issues, which can be helpful for potential guests.

2. **Accuracy and Depth**:
   - **Assistant A**: Accurate but slightly less detailed in terms of the variety of feedback.
   - **Assistant B**: More comprehensive by including both positive and negative feedback, giving a fuller picture of the guest experience.

3. **Creativity and Level of Detail**:
   - **Assistant A

## 대규모 평가 스크립트
- 원래는 많은 수의 다양한 데이터에 대해 평가를 하지만, 동일한 Prompt에 대해 temperature 높여서 평가 진행

In [64]:
# 같은 리뷰에 대해서 요약을 10번 수행(일반적으로는 50~100번 수행하면 좋음)
eval_count = 10

summaries_baseline = [summarize(reviews, PROMPT_BASELINE, temperature=1.0) for _ in range(eval_count)]
summaries_baseline

['경포해변을 품고 있는 강릉 숙소로, 깨끗하고 바다 뷰가 아름다웠으며, 직원들도 친절했다. 객실이 깔끔하고 편안하며, 위치도 좋아서 다시 방문하고 싶은 곳이었다.',
 '다양한 리뷰가 있지만, 숙소가 바닷가 바로 앞이어서 좋다는 점이 여러 리뷰에서 반복되고, 객실이 깨끗하고 친절한 서비스로 만족스러운 여행을 보내고자 하는 트렌드를 보입니다. 오션뷰와 뷰가 좋은 것도 많은 호평을 받았습니다.',
 '바닷가에 위치한 숙소로 친구들과 만족하며 이용했으며, 오션뷰와 쾌적한 객실이 좋았다. 현지 음식점과 근접한 위치가 편리하며, 룸 컨디션과 청결도가 우수했음을 강조했다. 위치, 깨끗함, 친절한 직원, 그리고 바다뷰가 좋았고 휴식을 취하기에 좋았다고 평가했다.',
 '숙소는 바다 바로 앞에 위치하고, 객실은 깨끗하며 수압 좋은 샤워기를 갖추고 있어서 친구들도 만족하며 완벽한 숙박을 했다는 리뷰를 요약하자면, "바닷가 숙소, 깨끗하고 수압 좋은 샤워기 갖춤, 친구들도 만족"으로 요약할 수 있겠다.',
 '바다 전망이 좋고 객실이 깨끗한 호텔. 매우 친절한 직원들이 좋은 서비스를 제공해주며, 바다가 보이는 객실을 추천하는 리뷰. 위치도 편리하고 만족스러움. 추천합니다.',
 '여러 숙박객들이 바다뷰와 객실의 청결함, 전망의 아름다움, 주변 편의시설에 만족하여 재방문 의사가 높음. 위치가 좋고 직원들도 친절하며 객실 관리가 철저했음을 언급함.',
 '바닷가 전망 최고, 깨끗하고 친절한 숙소. 오션뷰 예쁘고, 포근한 침대로 다음 방문 의사 100%! 하루밤 편안히 보내기 좋고, 위치와 청결이 만족스러웠다.',
 '바닷가에 위치한 숙소로 객실이 깨끗하고 수압이 좋으며, 오션뷰 객실에서는 바다와 호수를 감상할 수 있다. 직원들은 친절하고, 주차도 편리하며 뷰가 아름답다. 종합적으로 만족스러운 숙소라는 평이 많이 남아 있었다.',
 '해변이 가까워 바다 전망이 좋고 객실이 깨끗하며 친절한 직원이 있는 숙소.와 카운터 매너가 우수하고 해안뷰도 아름답다고 함.주차도 넓고 바다 소리 들

In [65]:
pairwise_words = [summary_4o_20250530 for _ in range(len(summaries_baseline))]
pairwise_words

['이 숙소는 바다와 가까운 위치와 아름다운 오션뷰로 많은 이용객들에게 호평을 받고 있습니다. 객실은 매우 청결하고, 침구와 화장실 상태도 좋다는 평가가 많습니다. 직원들의 친절한 서비스와 편리한 주차 시설도 긍정적인 리뷰를 받았습니다. 다만, 일부 이용객들은 밤에 소음이 있거나 난방이 부족하다는 점을 지적했습니다. 전반적으로 재방문 의사가 높고, 가성비 좋은 숙소로 추천받고 있습니다.',
 '이 숙소는 바다와 가까운 위치와 아름다운 오션뷰로 많은 이용객들에게 호평을 받고 있습니다. 객실은 매우 청결하고, 침구와 화장실 상태도 좋다는 평가가 많습니다. 직원들의 친절한 서비스와 편리한 주차 시설도 긍정적인 리뷰를 받았습니다. 다만, 일부 이용객들은 밤에 소음이 있거나 난방이 부족하다는 점을 지적했습니다. 전반적으로 재방문 의사가 높고, 가성비 좋은 숙소로 추천받고 있습니다.',
 '이 숙소는 바다와 가까운 위치와 아름다운 오션뷰로 많은 이용객들에게 호평을 받고 있습니다. 객실은 매우 청결하고, 침구와 화장실 상태도 좋다는 평가가 많습니다. 직원들의 친절한 서비스와 편리한 주차 시설도 긍정적인 리뷰를 받았습니다. 다만, 일부 이용객들은 밤에 소음이 있거나 난방이 부족하다는 점을 지적했습니다. 전반적으로 재방문 의사가 높고, 가성비 좋은 숙소로 추천받고 있습니다.',
 '이 숙소는 바다와 가까운 위치와 아름다운 오션뷰로 많은 이용객들에게 호평을 받고 있습니다. 객실은 매우 청결하고, 침구와 화장실 상태도 좋다는 평가가 많습니다. 직원들의 친절한 서비스와 편리한 주차 시설도 긍정적인 리뷰를 받았습니다. 다만, 일부 이용객들은 밤에 소음이 있거나 난방이 부족하다는 점을 지적했습니다. 전반적으로 재방문 의사가 높고, 가성비 좋은 숙소로 추천받고 있습니다.',
 '이 숙소는 바다와 가까운 위치와 아름다운 오션뷰로 많은 이용객들에게 호평을 받고 있습니다. 객실은 매우 청결하고, 침구와 화장실 상태도 좋다는 평가가 많습니다. 직원들의 친절한 서비스와 편리한 주차 시설도 긍정적

- 대규모 데이터를 비교평가

In [66]:

from tqdm import tqdm

def pairwise_eval_batch(reviews, answers_a, answers_b):
    a_cnt, b_cnt, draw_cnt = 0, 0, 0
    for i in tqdm(range(len(answers_a))):
        # 최종 결과
        verdict_text = pairwise_eval(reviews, answers_a[i], answers_b[i])

        # A인경우 카운트
        if '[[A]]' in verdict_text:
            a_cnt += 1
        # B인경우 카운트
        elif '[[B]]' in verdict_text:
            b_cnt += 1
        # C인경우 카운트
        elif '[[C]]' in verdict_text:
            draw_cnt += 1
        # 나머지
        else:
            print('Evaluation Error')
    # a good, b bad, c 무승부 비슷함.
    return a_cnt, b_cnt, draw_cnt

# 요약한 것과 대조군 비교
wins, losses, ties = pairwise_eval_batch(reviews, summaries_baseline, [summary_4o_20250530 for _ in range(len(summaries_baseline))])
print(f'Wins: {wins}, Losses: {losses}, Ties: {ties}')

100%|██████████| 10/10 [00:59<00:00,  5.97s/it]

Wins: 0, Losses: 10, Ties: 0


### 응답결과 고도화 1 - 요약 조건들 명시 후 평가
- role과 수행 목표 정의
- 요약 결과에 대한 충족 조건
    - 톤엔 매너와 예시
    - 요약 결과에 갯수

In [67]:
prompt = f"""당신은 요약 전문가입니다. 사용자 숙소 리뷰들이 주어졌을 때 요약하는 것이 당신의 목표입니다.

요약 결과는 다음 조건들을 충족해야 합니다:
1. 모든 문장은 항상 존댓말로 끝나야 합니다.
2. 숙소에 대해 소개하는 톤앤매너로 작성해주세요.
  2-1. 좋은 예시
    a) 전반적으로 좋은 숙소였고 방음도 괜찮았다는 평입니다.
    b) 재방문 예정이라는 평들이 존재합니다.
  2-2. 나쁜 예시
    a) 좋은 숙소였고 방음도 괜찮았습니다.
    b) 재방문 예정입니다.
3. 요약 결과는 최소 2문장, 최대 5문장 사이로 작성해주세요.
    
아래 숙소 리뷰들에 대해 요약해주세요:"""

eval_count = 10
summaries = [summarize(reviews, prompt, temperature=1.0) for _ in range(eval_count)]
wins, losses, ties = pairwise_eval_batch(reviews, summaries, [summary_4o_20250530 for _ in range(len(summaries))])
print(f'Wins: {wins}, Losses: {losses}, Ties: {ties}')

100%|██████████| 10/10 [00:44<00:00,  4.49s/it]

Wins: 2, Losses: 8, Ties: 0


### 응답결과 고도화 2 - 입력 데이터의 품질 증가
- 리뷰 길이가 짧은 것 필터링함, 고품질 리뷰만 남김

In [68]:
import datetime
from dateutil import parser

def preprocess_reviews(path='./res/1.gangneung_mare.json'):
    with open(path, 'r', encoding='utf-8') as f:
        review_list = json.load(f)

    reviews_good, reviews_bad = [], []

    current_date = datetime.datetime.now()
    date_boundary = current_date - datetime.timedelta(days=6*30)

    filtered_cnt = 0
    for r in review_list:
        review_date_str = r['date']
        try:
            review_date = parser.parse(review_date_str)
        except (ValueError, TypeError):
            review_date = current_date

        if review_date < date_boundary:
            continue

        # 고품질 리뷰만 남김
        if len(r['review']) < 30:
            filtered_cnt += 1
            # print(r['review'])
            continue

        if r['stars'] == 5:
            reviews_good.append('[REVIEW_START]' + r['review'] + '[REVIEW_END]')
        else:
            reviews_bad.append('[REVIEW_START]' + r['review'] + '[REVIEW_END]')

    # good, bad 각각의 리스트의 요소 최대 길이가 50까지 되도록 자름 
    # print("min(len(reviews_good), 50) : ", min(len(reviews_good), 50))
    # print("len(reviews_good): ", len(reviews_good))
    reviews_good = reviews_good[:min(len(reviews_good), 50)]
    reviews_bad = reviews_bad[:min(len(reviews_bad), 50)]
    # print(len(reviews_good))
    # print(reviews_good)

    reviews_good_text = '\n'.join(reviews_good)
    reviews_bad_text = '\n'.join(reviews_bad)

    return reviews_good_text, reviews_bad_text

reviews, _ = preprocess_reviews()

In [69]:
# 평가 테스트
eval_count = 10
summaries = [summarize(reviews, prompt, temperature=1.0, model='gpt-3.5-turbo-0125') for _ in range(eval_count)]
wins, losses, ties = pairwise_eval_batch(reviews, summaries, [summary_4o_20250530 for _ in range(len(summaries))])
print(f'Wins: {wins}, Losses: {losses}, Ties: {ties}')

100%|██████████| 10/10 [00:41<00:00,  4.18s/it]

Wins: 3, Losses: 7, Ties: 0


### 응답결과 고도화 3 - One-Shot Prompting

In [71]:
reviews_1shot, _ = preprocess_reviews('./res/1.gangneung_mare.json')
summary_1shot = summarize(reviews_1shot, prompt, temperature=0.0, model='gpt-3.5-turbo-0125')
prompt_1shot = f"""당신은 요약 전문가입니다. 사용자 숙소 리뷰들이 주어졌을 때 요약하는 것이 당신의 목표입니다.

요약 결과는 다음 조건들을 충족해야 합니다:
1. 모든 문장은 항상 존댓말로 끝나야 합니다.
2. 숙소에 대해 소개하는 톤앤매너로 작성해주세요.
  2-1. 좋은 예시
    a) 전반적으로 좋은 숙소였고 방음도 괜찮았다는 평입니다.
    b) 재방문 예정이라는 평들이 존재합니다.
  2-2. 나쁜 예시
    a) 좋은 숙소였고 방음도 괜찮았습니다.
    b) 재방문 예정입니다.
3. 요약 결과는 최소 2문장, 최대 5문장 사이로 작성해주세요.

다음은 리뷰들과 요약 예시입니다.
예시 리뷰들:
{reviews_1shot}
예시 요약 결과:
{summary_1shot}
    
아래 숙소 리뷰들에 대해 요약해주세요:"""

summaries = [summarize(reviews, prompt, temperature=1.0, model='gpt-3.5-turbo-0125') for _ in range(eval_count)]
wins, losses, ties = pairwise_eval_batch(reviews, summaries, [summary_4o_20250530 for _ in range(len(summaries))])
print(f'Wins: {wins}, Losses: {losses}, Ties: {ties}')

100%|██████████| 10/10 [00:41<00:00,  4.18s/it]

Wins: 5, Losses: 5, Ties: 0


In [ ]:
summaries

['강릉 여행길에 들렀는데 아주 예약을 잘한 숙소입니다. 호텔이 주인의 품격이 느껴지며 조용하고 좋습니다. 로비도 고급지고 객실도 깔끔해서 너무 만족스러운 1박이었습니다. 침구가 너무 좋아서 편하게 잠을 잘 수 있었고, 다음에도 여행을 가면 여기에 다시 묵을 것입니다. 부모님도 예약해 드렸는데 매우 만족하셨고, 객실은 깨끗하고 아늑하며 침구류 냄새가 나지 않아 좋았습니다. 위치가 좋아 주변 인프라가 잘 구축되어 있지만, 외부 음식 관련 안내가 필요할 것 같습니다. 프론트 데스크 직원도 친절하고 방도 깨끗하여 편안한 시간을 보낼 수 있었고, 조식 샌드위치가 맛있어서 좋았습니다. 전체적으로 따뜻하고 깨끗하여 다음에도 이용할 것 같습니다.',
 '강릉 여행길에 들르게 되어 예약을 잘한 호텔이었습니다. 주인의 품격을 느낄 수 있었고 조용하고 좋은 분위기였습니다. 로비와 객실이 모두 고급스럽고 깔끔하며 침구도 편안하여 만족스러운 1박을 보냈습니다. 친절한 안내와 정리정돈으로 편안한 휴식을 취할 수 있었고, 부모님도 너무 만족하셨습니다. 전체적으로 깨끗하고 아늑한 숙소로 외부 음식 관련 안내가 더 필요하다는 의견이 있지만, 프론트와 직원분들의 친절한 서비스를 감상할 수 있었습니다.',
 '강릉 여행길에 들르게 되어 아주 잘 예약한 호텔이었습니다. 주인의 품격이 느껴지며 조용하고 좋았어요. 로비도 고급스럽고 객실도 깔끔해 만족스러운 1박을 했습니다. 침구가 훌륭해서 편안한 휴식을 취할 수 있었고, 다음 여행 때 또 방문할 예정입니다. 친절한 안내와 청결한 정리 정돈으로 편안하게 쉴 수 있었고, 부모님도 너무 만족하셨어요. 또한, 객실이 깨끗하고 아늑하여, 침구류의 냄새도 거의 느껴지지 않아 좋은 경험을 했습니다. 객실 청결이 훌륭하며 주변 인프라가 우수하여 편안한 숙박을 즐겼지만, 외부 음식 관련 사전 안내가 필요한 부분이 있습니다. 호텔의 시설이 깔끔하고 택지지구에 위치해 주변 환경이 좋았으며, 프론트 데스크 직원도 친절하고 방도 깨끗해 편안한 휴식을 취할 수 있었습니다

### 응답결과 고도화 3 - one-Shot Prompting

In [ ]:
prompt_1shot = f"""당신은 요약 전문가입니다. 사용자 숙소 리뷰들이 주어졌을 때 요약하는 것이 당신의 목표입니다. 다음은 리뷰들과 요약 예시입니다.
예시 리뷰들:
{reviews_1shot}
예시 요약 결과:
{summary_1shot}
    
아래 숙소 리뷰들에 대해 요약해주세요:"""

summaries = [summarize(reviews, prompt_1shot, temperature=1.0, model='gpt-3.5-turbo-0125') for _ in range(eval_count)]
wins, losses, ties = pairwise_eval_batch(reviews, summaries, [summary_4o_20250530 for _ in range(len(summaries))])
print(f'Wins: {wins}, Losses: {losses}, Ties: {ties}')

100%|██████████| 10/10 [00:46<00:00,  4.61s/it]

Wins: 10, Losses: 0, Ties: 0


### prompt_1shot을 피클파일로 저장하기

In [72]:
# prompt_1shot을 피클파일로 저장하기
file_name = "./res/prompt_1shotv1.pickle"
# 파일로 저장
import pickle
with open(file_name, 'wb') as f:
    pickle.dump(prompt_1shot, f)

In [73]:
print(summaries)

['전반적으로 좋은 편인데, 숙소 위치가 정말 좋습니다. 바닷가 바로 앞이라 산책하기도 좋고, 근처에 맛집이 많아서 식사하기도 편리했습니다. 객실 청결 상태도 매우 좋았고, 직원분들도 친절하여 편안한 휴식을 취할 수 있었습니다. 다음에도 꼭 다시 방문하고 싶은 숙소입니다. 별로 단점은 없으나 티비 사용 시 넷플릭스가 끊기는 문제 있음. 결과적으로, 다시 방문 의사 100% 있습니다.', '객실이 깨끗하고 샤워기 수압이 좋고 물도 따뜻하게 잘 나오는 좋은 숙소입니다. 다들 만족하고 갑니다.\n오션뷰가 정말 좋았고 바다를 바라보며 편안히 먹었고, 새벽에 바다 구경까지 했습니다. 잘 보내다 왔어요.\n황금연휴에 방문한 숙소는 오션뷰가 아름다웠고, 직원분들의 친절함도 기분 좋게 했습니다. 다음에도 꼭 다시 방문하고 싶네요.\n방음도 좋고 깨끗하며 주차하기도 편리한 좋은 숙소였습니다. 위치가 좋아서 이동이 편리합니다. 객실과 화장실도 매우 청결했습니다.결혼기념일로 머물렀는데 너무 편안하게 쉬고, 침대가 푹신하고 전망도 좋았습니다.친절한 24시간 데스크와 깔끔한 오션뷰, 깨끗한 객실 내부를 갖추고 있습니다. 다음에도 이용하고 싶은 숙소입니다.', '전반적으로 좋은 숙소로, 바닷가에 위치해 있어 친구들과 함께 방문했다가 만족하고 갑니다. 샤워기 수압이 좋고, 깨끗한 객실을 제공해 편안하게 묵었습니다. 오션뷰를 즐기며 좋은 추억을 만들었고, 침구와 객실 청결도에 대해서도 만족했습니다. 다음에도 꼭 다시 방문할 의사가 있습니다. 객실은 깔끔하고 편안했으며, 오션뷰가 예쁘게 보였습니다. 직원분들도 친절하고, 내부 시설들도 만족스러웠습니다. TV 문제를 제외하고 전반적으로 좋은 경험했습니다. 위치가 좋아 바다와 호수를 즐길 수 있었으며, 깨끗한 객실과 친절한 24시간 데스크를 갖추고 있습니다. 다음에도 이용하고 싶은 숙소입니다.환경이 좋고 청결한 상태를 유지하며 좋은 알바도 좋고, 테라스를 즐길 수 있는 곳입니다. 부족한 점이라면 담배 냄새가 올라오는 것이었지만 전체적으로 만족하